### Automatic Differentiation

Automatic differentiation is a technique for numerically computing derivatives efficiently. The motivation for developing this technique arose from the complexity of computing gradients of functions with many input variables or many outputs.

For the following discussion consider a function $f: \mathbb{R}^n \rightarrow \mathbb{R}^m$ (that is, $n$ input variables and $m$ outputs).

**Manual differentiation** of functions is of course infeasible for complicated functions. Exact analytic differentiation using computers (following strictly the mathematical rules of differentiation, like in the manual manner) is possible, and is called **symbolic differentiation**. However, for complicated functions this option quickly becomes inefficient as well, due to the exponential overhead caused by computing derivatives of nested terms (consider a product term with derivative $(uv)' = u'v + v'u$, for example). It seems that **numerical differentiation** using the [finite-difference method](https://en.wikipedia.org/wiki/Finite_difference_method) might be an efficient candidate. Still, this method's complexity scales linearly with both $m$ and $n$ - so if $m$ or $n$ are very large, that's a problem.

**Automatic differentiation** is a sort of symbloic-numerical technique that for some cases provides better results than the former-mentioned methods. The concept of this method is based on a reconstruction of the given function $f$ with intermidiate variables called **primals**. The reconstruction process is being done "bottom-up" - we start from defining $n$ primals for the $n$ different variables. Then we look for typical patterns in $f$'s structure to define the next primals. We keep defining primals until the function is completly expressed with primals. This description might sound a little tedious, therefore consider the following demonstration; Given a function $f: \mathbb{R}^2 \rightarrow \mathbb{R}$:

$$f(x1, x2) = e^{x1} - x1x2 + cos(x1x2) - sin(e^{x1})$$

We define the following primals:
$$ p_1 = x_1 $$ 
$$ p_2 = x_2 $$
$$ p_3 = e^{p_1} $$
$$ p_4 = p_1p_2 $$
$$ p_5 = p_3 - p_4 $$
$$ p_6 = cos(p_4) $$
$$ p_7 = sin(p_3) $$
$$ p_8 = p_6 - p_7 $$
$$ p_9 = p_5 + p_8 $$

Now, given these primals + a numerical data point $x = (x1, x2)$ to evaluate, one can compute the exact analytic value (excluding imprecisions imposed by the binaric nature of computers) of $f$'s partial derivatives at $x$, using auto-diff. We continue with our example by setting $x = (1,1)$ and computing $\frac{\partial f}{\partial x_1}$ at $x$.

From here, there are 2 options to proceed - Either using **forward auto-diff** or **reverse auto-diff**. We begin by demonstrating forward auto-diff; The method is to evaluate the value of each primal and its partial derivative (each primal's partial derivative is called a **tangent**) step-by-step. Remember that for now we evaluate $\frac{\partial f}{\partial x_1}$ at $x = (1,1)$, therefore (primals on the left, tangents on the right):

$$ p_1 = x_1 = 1 \ | \ \ \dot{p_1} = 1$$

$$ p_2 = x_2 = 1 \ | \ \ \dot{p_2} = 0$$

$$ p_3 = e^{p_1} = e^{1} = e \ | \ \ \dot{p_3} = e^{p_1} = e$$

$$ p_4 = p_1p_2 = 1 \ | \ \ \dot{p_4} = b\dot{p_1} = 1$$

$$ p_5 = p_3 - p_4 = e - 1 \ | \ \ \dot{p_5} = \dot{p_3} - \dot{p_4} = e - 1$$

$$ p_6 = cos(p_4) = cos(1) \ | \ \ \dot{p_6} = -sin(p_4)\dot{p_4} = -sin(1)$$

$$ p_7 = sin(p_3) = sin(e) \ | \ \ \dot{p_7} = cos(p_3)\dot{p_3} = cos(e) \times e $$

$$ p_8 = p_6 - p_7 = cos(1) - sin(e) \ | \ \ \dot{p_8} =  \dot{p_6} - \dot{p_7} = - sin(1) - cos(e) \times e$$

$$ p_9 = p_5 + p_8 = e - 1 + cos(1) - sin(e) = 1.848 \ | \ \ \dot{p_9} =  \dot{p_5} + \dot{p_8} = e - 1 - sin(1) - cos(e) \times e = 3.355$$

To check the answers above, it's time to introduce **JAX**. JAX is a python library that offers (among other things) auto-diff capabilities. JAX has a module named `numpy` that provides an API which is similar to the original NumPy's API, that one should use in order to exploit JAX's capabilities, like auto-diff. Therefore we begin by importing the necessary modules and defining $f$ using JAX's API:

In [1]:
import numpy as np
import jax.numpy as jnp

f = lambda x1, x2: jnp.e**x1 - x1*x2 + jnp.cos(x1*x2) - jnp.sin(jnp.e**x1)

First, we simply evaluate $f(1,1)$ to check that our evaluation for the primal $p_9$ above is correct:

In [11]:
print(f(1., 1.))

1.8478026


Correct!

Now we use the function `grad` provided by JAX, in order to verify our result for $\frac{\partial f}{\partial x_1}$ at $x = (1,1)$ (that is $\dot{p_9} = 3.355$). `grad` gets a function as an input, returns another function with the same arguments that computes the gradient of the original function, using auto-diff. Therefore:

In [3]:
from jax import grad

grad_f = grad(f, argnums=0)
print(grad_f(1., 1.))

# argnums=0 means "evaluate the partial derivative of x1".
# argnums=1 means "evaluate the partial derivative of x2".
# argnums=[0,1] means "evaluate both partial derivatives".

3.3551602


Correct!

Note that by evaluating also $\frac{\partial f}{\partial x_2}$ at $x = (1,1)$ we obtain the full Jacobian-matrix of $f$ at $x = (1,1)$. In the case of $f$ the Jacobian-matrix is a $1 \times 2$ matrix (In the general case it is an $m \times n$ matrix, just like the map $f$). The Jacobian-matrix essentially contains the gradients of each output in each row (stretched horizontally). We can use the `jacfwd` (Jacobian-Forward) provided by JAX to demonstrate that:

In [4]:
from jax import jacfwd

jacfwd(f, argnums=[0, 1])(1., 1.)

(DeviceArray(3.3551602, dtype=float32, weak_type=True),
 DeviceArray(-1.841471, dtype=float32, weak_type=True))

Forward auto-diff allows computing of all $m$ outputs in a single "pass" through the algorithm, by reusing primals and tangents for multiples outputs, as much as possible. However, as aforementioned, a single "forward-pass" yields partial derivatives only for 1 input variable, out of $n$ input variables. In short, in a single "forward-pass" one obtains a column of $f$'s Jacobian-matrix (for a specific input data $x$). I.e, forward auto-diff is good for the case of a few inputs and many outputs, that is $n \lt\lt m$. In the opposite case $m \lt\lt n$ - the case of many inputs and a few outputs, forward auto-diff is inefficient.

Luckily, **Reverse auto-diff** is perfectly suited for that case. The technique of reverse auto-diff is implemented as follows; First, we reconstruct $f$ with primals, in the exact manner like in forward auto-diff, but without computing tangents. That is, for our $f$:

$$f(x1, x2) = e^{x1} - x1x2 + cos(x1x2) - sin(e^{x1})$$

$$ p_1 = x_1 = 1$$

$$ p_2 = x_2 = 1$$

$$ p_3 = e^{p_1} = e^{1} = e$$

$$ p_4 = p_1p_2 = 1$$

$$ p_5 = p_3 - p_4 = e - 1$$

$$ p_6 = cos(p_4) = cos(1)$$

$$ p_7 = sin(p_3) = sin(e)$$

$$ p_8 = p_6 - p_7 = cos(1) - sin(e)$$

$$ p_9 = p_5 + p_8 = e - 1 + cos(1) - sin(e) = 1.848$$

Then, we apply a technique of reverse computation of **adjoints**, which defined for each primal as follows ($\overline{p_i}$ is the adjoint of the primal $p_i$, $P$ is the set of all adjoints and $|P|$ is the cardinality of $P$, i.e the total number of primals):

$$ \overline{p_i} =  \sum_{j = i + 1}^{| P\ |}\overline{p_j}{\frac{\partial{p_j}}{\partial{p_i}}} $$

We set the value of last adjoint to 1, that is $p_{|P|} = 1$ , and from there we compute all adjoints backwards toward the first primals, which are the input variables. The values of the first primals' adjoints are the partial derivatives, i.e the complete gradient of a single output is obtained.

In our function $f$ there is a single output ($m = 1$) so the full gradient is obtained by a single "reverse-pass":

$$ \overline{p_9} = 1 $$

$$ \overline{p_8} = 1 $$

$$ \overline{p_7} = -1 $$

$$ \overline{p_6} = 1 $$

$$ \overline{p_5} = 1 $$

$$ \overline{p_4} = \overline{p_5} \times (-1) + \overline{p_6} \times (-sin(p_4)) = -1 -sin(1) = -1.841 $$

$$ \overline{p_3} = \overline{p_5} \times (1) + \overline{p_7} \times cos(p_3) = 1 - cos(e) = 1.912 $$

$$ \overline{p_2} = \overline{p_4} \times p1 = -1.841 $$

$$ \overline{p_1} = \overline{p_3} \times e^{p_1} + \overline{p_4} \times p2 = 1.912e -1.841 = 3.356 $$

Correct! We have obtained $\overline{p_1} = \frac{\partial{f}}{\partial{x_2}} = -1.841$ and $\overline{p_2} = \frac{\partial{f}}{\partial{x_1}} = 3.356$ in a single "reverse-pass".

As a matter of fact, the `grad` function we have used before actually uses reverse auto-diff.

JAX offers a function called `jacrev` (Jacobian-Reverse) that does exactly what we have done manually above, to be used as follows:

In [5]:
from jax import jacrev

jacrev(f, argnums=[0, 1])(1. ,1.)

(DeviceArray(3.3551602, dtype=float32, weak_type=True),
 DeviceArray(-1.841471, dtype=float32, weak_type=True))

### JVP, VJP

Using the same techniques of forward and reversed auto-diff, It's possible to efficiently compute with JAX a product of a Jacobian matrix with a vector from the right (`jvp` = Jacobian-Vector Product) or from the left (`vjp` = Vector-Jacobian Product), While `jvp` uses forward auto-diff and `vjp` uses reversed auto-diff.

To demonstrate that, we define another function $g: \mathbb{R}^3 \rightarrow \mathbb{R}^2$, and a sample data point $x = (1, 1, 1)$:

In [6]:
def g(x):
    return jnp.array([x[0]**3 + x[1]**2 - x[2] + 5,
                     4*x[0]**3 - 7*x[1]**2 + 5*x[2]])

x = jnp.array([1., 1., 1.])

Now, we define a vector $v_r$ to evaluate with `jvp` and execute:

In [7]:
from jax import jvp

v_r = jnp.array([1., 2., 3.])

print(jvp(g, primals=(x,), tangents=(v_r,)))

(DeviceArray([6., 2.], dtype=float32), DeviceArray([ 4., -1.], dtype=float32))


We have obtained a tuple with 2 records - The first record is just the value of $g(x)$. The second record is the Jacobian-vector product of $g$'s Jacobian-matrix with $v_r$ (from the right). We verify that as follows:

In [8]:
print(f"g(1, 1 ,1) = {g(x)}")
print(f"jvp(g, ({x}), ({v_r})) = {jacfwd(g)(x) @ v_r}")

g(1, 1 ,1) = [6. 2.]
jvp(g, ([1. 1. 1.]), ([1. 2. 3.])) = [ 4. -1.]


Correct!

So what is the point in `jvp` if we could just compute the Jacobian-matrix using regular forward auto-diff and multiply? By using `jvp` the calculations are being made seemlessly along the process, without ever even computing the full Jacobian-matrix - which in our case is pretty humble, but in the general case $m$ and $n$ can grow arbitrarly. In some cases it would be impossible to even store the Jacobian-matrix, due to its size. So clearly obtaining a desired result using `jvp` is preferable, and indeed it performs much faster as the dimensions of the Jacobian-matrix scale.

`vjp` exploits the same logic as `jvp`, but uses reverse auto-diff. So we define a vector $v_l$ to evaluate with `vjp` and execute:

In [9]:
from jax import vjp

v_l = jnp.array([1., 2.]).T

g_x, vjp_fun = vjp(g, x)

print(g_x)
print(vjp_fun)

[6. 2.]
Partial(<jax._src.tree_util._HashableCallableShim object at 0x7f1ce02c6be0>, Partial(<jax._src.tree_util._HashableCallableShim object at 0x7f1ce0240b50>, ()))


Again, We have obtained a tuple with 2 records - The first record is just the value of $g(x)$. The second record is a function that computes a vector-Jacobian product of $g$'s Jacobian-matrix with $v_l$ (from the left). We verify that as follows:

In [10]:
print(vjp_fun(v_l)[0])
print(v_l @ jacrev(g)(x))

[ 27. -26.   9.]
[ 27. -26.   9.]


Correct!